In [12]:

import torch
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/judd/Documents/optimisation_of_superconduncting_circuits/core')
from fluxonium import Fluxonium 
import general as general
import scqubits as sc


In [13]:
#comparing H calculations

EJ = 10
EC = 2.5
EL = 1.9
flux = 0.5
dim = 110

fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'auto_H')
fluxonium2 = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

general.t2(fluxonium2, fluxonium2.t1_supported_noise_channels(), fluxonium2.tphi_supported_noise_channels()).item()-general.t2(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels()).item()


0.03870505793020129

In [14]:
#compare my pytorch computations to scqubits

fluxonium_sc = sc.Fluxonium(
            EJ = 10,
            EC = 2.5,
            EL = 1.9,
            flux = 0.5,
           cutoff = 110)

print(fluxonium_sc.t1_quasiparticle_tunneling() - 1/general.effective_t1_rate(fluxonium2 , "t1_quasiparticle_tunneling").item())
print(fluxonium_sc.t1_charge_impedance() - 1/general.effective_t1_rate(fluxonium2 , "t1_charge_impedance").item())
print(fluxonium_sc.t1_capacitive() - 1/general.effective_t1_rate(fluxonium2 , "t1_capacitive").item())
print(fluxonium_sc.t1_inductive() - 1/general.effective_t1_rate(fluxonium2 , "t1_inductive").item())
print(fluxonium_sc.t1_flux_bias_line() - 1/general.effective_t1_rate(fluxonium2 , "t1_flux_bias_line").item())

print(fluxonium_sc.tphi_1_over_f_cc() - 1/general.effective_tphi_rate(fluxonium2 , "tphi_1_over_f_cc").item())
print(fluxonium_sc.tphi_1_over_f_flux() - 1/general.effective_tphi_rate(fluxonium2, "tphi_1_over_f_flux()").item())

2.18118773586878e+37
-4.803859017261075
-40847.98916019022
-166662.5068125301
-317168.7211345257
-0.046930446987971663
4.291329747424885e+16


In [16]:
#TENSORS

#INPUT

#random values
EJ = torch.rand(1, requires_grad=True, dtype=torch.double)
EC = torch.rand(1, requires_grad=True, dtype=torch.double)
EL = torch.rand(1, requires_grad=True, dtype=torch.double)
flux = torch.tensor([0.5], requires_grad=True, dtype=torch.double)

EJ.data = EJ.data * (20 - 2.5) + 2.5
EC.data = EC.data * (8 - 1e-3) + 1e-3
EL.data = EL.data * (10 - 2e-1) + 2e-1

# PARAMETERS
dim = 110
learn_rate = 3e4

# BOUNDS
EJ_bounds = [2.5, 150]
EC_bounds = [1e-3, 8]
EL_bounds = [2e-1, 10]
flux_bounds = [0,1]

# RECORDING VALUES

EJ_vals = [EJ.item()]
EC_vals = [EC.item()]
EL_vals = [EL.item()]
cost_vals = []

# GRADIENT DESCENT

max_T2 = 0

for i in range(0,700):

    #reinstantiate values
    EJ = torch.tensor(EJ.item(), requires_grad=True, dtype=torch.double)
    EC = torch.tensor(EC.item(), requires_grad=True, dtype=torch.double)
    EL = torch.tensor(EL.item(), requires_grad=True, dtype=torch.double)
    flux = torch.tensor(flux.item(), requires_grad=True, dtype=torch.double)
    fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

    T2 = general.t2(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())

    if T2 > max_T2:
        max_T2 = T2
    
    #autograd on objective function
    cost = 1/T2
    cost.backward()

    cost_vals.append(cost.item())

    print("CYCLE: ",i)
    print("T = ",T2.item(), ", cost = ", cost.item(), ", Best T = ", max_T2.item())
    print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())
    print("change EJ,EC,EL,flux = ",EJ.grad.item()*learn_rate,EC.grad.item()*learn_rate,EL.grad.item()*learn_rate,flux.grad.item()*learn_rate)
    
    #update parameters according to gradient
    with torch.no_grad():
        # Near the boundary, we project the gradient onto it. This is the typical approach to constraints.
        if EJ - EJ.grad*learn_rate > EJ_bounds[0] and EJ.data - EJ.grad*learn_rate < EJ_bounds[1]:
            EJ -= EJ.grad*learn_rate
        elif EJ - EJ.grad*learn_rate <= EJ_bounds[0]:
            EJ += (torch.tensor(EJ_bounds[0]) - EJ)
        else:
            EJ += (torch.tensor(EJ_bounds[1]) - EJ)
        EJ_vals.append(EJ.item())
        
        if EC - EC.grad*learn_rate > EC_bounds[0] and EC - EC.grad*learn_rate < EC_bounds[1]:
            EC -= EC.grad*learn_rate
        elif EC - EC.grad*learn_rate <= EC_bounds[0]:
            EC += (torch.tensor(EC_bounds[0]) - EC)
        else:
            EC += (torch.tensor(EC_bounds[1]) - EC)
        EC_vals.append(EC.item())
        
        if EL - EL.grad*learn_rate > EL_bounds[0] and EL - EL.grad*learn_rate < EL_bounds[1]:
            EL -= EL.grad*learn_rate
        elif EL - EL.grad*learn_rate <= EL_bounds[0]:
            EL += (torch.tensor(EL_bounds[0]) - EL)
        else:
            EL += (torch.tensor(EL_bounds[1]) - EL)
        EL_vals.append(EL.item())

    EJ.grad = None
    EC.grad = None
    EL.grad = None
    flux.grad = None

#append last cost value to cost array

fluxonium = Fluxonium(EJ, EC, EL, flux, dim, 'create_H')

T2 = general.t2(fluxonium, fluxonium.t1_supported_noise_channels(), fluxonium.tphi_supported_noise_channels())

cost = 1/T2
print("CYCLE: END")
print("T = ",T2.item(), ", cost = ", cost.item(), ", Best T = ", max_T2.item())
print("EJ,EC,EL,flux = ",EJ.item(),EC.item(),EL.item(),flux.item())

cost_vals.append(cost.item())


 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/fluxonium.py: 66UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/fluxonium.py: 26UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/fluxonium.py: 48UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 /Users/judd/Documents/optimisation_of_superconduncting_circuits/core/fluxonium.py: 58

CYCLE:  0
T =  583876.800904403 , cost =  1.712690071691559e-06 , Best T =  583876.800904403
EJ,EC,EL,flux =  6.412877624110612 6.239748558253344 3.1028848075770856 0.5
change EJ,EC,EL,flux =  -0.0021038417480408033 1.148698782024767e-11 3.0990024910585385e-14 291.4812373891813
CYCLE:  1
T =  583927.107140694 , cost =  1.7125425207551726e-06 , Best T =  583927.107140694
EJ,EC,EL,flux =  6.414981465858653 6.239748558241857 3.1028848075770545 0.5
change EJ,EC,EL,flux =  -0.0021042010687052056 1.1489596629971649e-11 3.101308908675307e-14 291.63668077173304
CYCLE:  2
T =  583977.4392346698 , cost =  1.7123949194176878e-06 , Best T =  583977.4392346698
EJ,EC,EL,flux =  6.417085666927358 6.239748558230367 3.1028848075770235 0.5
change EJ,EC,EL,flux =  -0.0021045603769703535 1.149220645740303e-11 3.1036175526451305e-14 -291.7922014484063
CYCLE:  3
T =  584027.7972015803 , cost =  1.71224726766703e-06 , Best T =  584027.7972015803
EJ,EC,EL,flux =  6.419190227304329 6.239748558218875 3.10288480

KeyboardInterrupt: 